In [4]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [98]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

import warnings
warnings.filterwarnings("ignore")

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [99]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [168]:
# Your code
from sklearn.model_selection import train_test_split
X = data["text"]
y = data["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Data Preprocessing

In [169]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


In [170]:
X.head()

0    DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...
1                                             Will do.
2    Nora--Cheryl has emailed dozens of memos about...
3    Dear Sir=2FMadam=2C I know that this proposal ...
4                                                  fyi
Name: text, dtype: object

## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [171]:
import re
import html
from bs4 import BeautifulSoup

def remove_inline_css_js(text: str) -> str:

    # Remove inline JavaScript/CSS and strip HTML to plain text.
    # Works with both escaped tags (&lt;tag&gt;) and real tags (<tag>).

    if not isinstance(text, str):
        return ""

    # If the input contains &lt;...&gt; entities, convert them to real <...> first.
    text = html.unescape(text)

    # 1) Remove <script>...</script> blocks
    text = re.sub(r'<script[^>]*>.*?</script>', '', text, flags=re.DOTALL | re.IGNORECASE)

    # 2) Remove <style>...</style> blocks
    text = re.sub(r'<style[^>]*>.*?</style>', '', text, flags=re.DOTALL | re.IGNORECASE)

    # 3) Remove inline CSS attributes: style="..."
    text = re.sub(r'\s*style="[^"]*"', '', text)

    # 4) Remove HTML comments <!-- ... -->
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)

    # 5) Strip the remaining HTML tags (<div>, <p>, <br>, etc.)
    text = BeautifulSoup(text, "html.parser").get_text()

    # 6) Remove leftover broken tag fragments like "STRONG>" or "/DIV>"
    text = re.sub(r'/?[A-Za-z]+>', '', text)

    # 7) Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# Example: apply to the X_test and X_train data

X_test = X_test.fillna("").astype(str).apply(remove_inline_css_js) 
X_train = X_train.fillna("").astype(str).apply(remove_inline_css_js)

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [172]:
X_train[0]

"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL I AM MIKE CHUKWU , THE MANAGER, BILLS AND EXCHANGE AT THE FOREIGN REMITTANCE DEPARTMENT OF THE ZENITH INTERNATIONAL BANK PLC. I AM WRITING THIS LETTER TO ASK FOR YOUR SUPPORT AND COOPERATION TO CARRY OUT THIS BUSINESS OPPORTUNITY IN MY DEPARTMENT. WE DISCOVERED AN ABANDONED SUM OF $15,000,000.00 (FIFTEEN MILLION UNITED STATES DOLLARS ONLY) IN AN ACCOUNT THAT BELONGS TO ONE OF OUR FOREIGN CUSTOMERS WHO DIED ALONG WITH HIS ENTIRE FAMILY OF A WIFE AND TWO CHILDREN IN NOVEMBER 1997 IN A PLANE CRASH. SINCE WE HEARD OF HIS DEATH, WE HAVE BEEN EXPECTING HIS NEXT-OF-KIN TO COME OVER AND PUT CLAIMS FOR HIS MONEY AS THE HEIR,BECAUSE WE CANNOT RELEASE THE FUND FROM HIS ACCOUNT UNLESS SOMEONE APPLIES FOR CLAIM AS THE NEXT-OF-KIN TO THE DECEASED AS INDICATED IN OUR BANKING GUIDELINES. UNFORTUNATELY, NEITHER THEIR FAMILY MEMBER NOR DISTANT RELATIVE HAS EVER APPEARED TO CLAIM THE SAID FUND. UPON THIS DISCOVERY,I AND OTHER OFFICIALS IN MY DEPARTMENT HAVE

In [191]:
# Your code
#############################################################
# Remove all the special characters
import string

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

X_train = X_train.apply(remove_punctuation)
X_test = X_test.apply(remove_punctuation)

#############################################################
# Remove numbers
def remove_numbers(text):
    return re.sub(r"\d+", "", text)                 # one or more digits

X_train = X_train.apply(remove_numbers)
X_test = X_test.apply(remove_numbers)

#############################################################
# Remove all single characters
def remove_single_chars(text):
    return re.sub(r"(?<=\s).(?=\s)", "", text)      # exactly one single character, if there is a space before and after this character
    # return re.sub(r"\b[a-zA-Z]\b", "", text)      # exactly one single letter

X_train = X_train.apply(remove_single_chars)
X_test = X_test.apply(remove_single_chars)

#############################################################
# Remove single characters from the start
def remove_first_char(text):
    return re.sub(r"^(?=\S\s)\S", "", text)         # remove first char if it is followed by a space

X_train = X_train.apply(remove_first_char)
X_test = X_test.apply(remove_first_char)

#############################################################
# Substitute multiple spaces with single space

def normalize_spaces(text):
    return re.sub(r"\s+", " ", text).strip()        # replace multiple spaces by only a single space. .striß -> Delete spaces at the beginning or the end

X_train = X_train.apply(normalize_spaces)
X_test = X_test.apply(normalize_spaces)

#############################################################
# Remove prefixed 'b'
def remove_b_before_capital(text):
    return re.sub(r"b(?=[A-Z])", "", text)          # delete "b" it is followed by a capital letter

X_train = X_train.apply(remove_b_before_capital)
X_test = X_test.apply(remove_b_before_capital)

#############################################################
# Convert to Lowercase
def to_lowercase(text):
    return text.lower()

X_train = X_train.apply(to_lowercase)
X_test = X_test.apply(to_lowercase)

X_train[0]

'dear sir strictly private business proposal mike chukwu manager bills exchange foreign remittance department zenith international bank plc writing letter ask support cooperation carry business opportunity department discovered abandoned sum fifteen million united states dollars account belongs one foreign customers died along entire family wife two children november plane crash since heard death expecting nextofkin come put claims money heirbecause cannot release fund account unless someone applies claim nextofkin deceased indicated banking guidelines unfortunately neither family member distant relative ever appeared claim said fund upon discoveryi officials department agreed make business release total amount account heir fund since one came discovered maintained account bank otherwise fund returned banks treasury unclaimed fund agreed ratio sharing stated thus foreign partner us officials department settlement local foreign expences incurred us course business upon successful comple

## Now let's work on removing stopwords
Remove the stopwords.

In [201]:
# Your code
from nltk.corpus import stopwords

def remove_words(text):
    stop_words = set(stopwords.words('english'))
    pattern = r"\b(" + "|".join(map(re.escape, stop_words)) + r")\b"
    return re.sub(pattern, "", text, flags=re.IGNORECASE)

X_train = X_train.apply(remove_words)
X_test = X_test.apply(remove_words)

X_train = X_train.apply(normalize_spaces)       # delete double spaces again
X_test = X_test.apply(normalize_spaces)       # delete double spaces again

X_train[0]

'dear sir strictly private business proposal mike chukwu manager bills exchange foreign remittance department zenith international bank plc writing letter ask support cooperation carry business opportunity department discovered abandoned sum fifteen million united states dollars account belongs one foreign customers died along entire family wife two children november plane crash since heard death expecting nextofkin come put claims money heirbecause cannot release fund account unless someone applies claim nextofkin deceased indicated banking guidelines unfortunately neither family member distant relative ever appeared claim said fund upon discoveryi officials department agreed make business release total amount account heir fund since one came discovered maintained account bank otherwise fund returned banks treasury unclaimed fund agreed ratio sharing stated thus foreign partner us officials department settlement local foreign expences incurred us course business upon successful comple

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [207]:
X_train.head(20)

29     regards mr nelson smithkindly reply private em...
535           able reach oscar supposed send pdb receive
695    huma abedin bim checking pat work jack jake re...
557                          announced monday cant today
836    bank africaagence san pedro bp san pedro cote ...
596    dear friendmy proposal surprising personal con...
165    soneri bank limitedsilver jubilee centercbritt...
918            easy joy powerful yahoo bookmarks toolbar
495    amrecollince addoattn ai need urgent assistanc...
824    dear nancyi much want meet weve trying pin sch...
65     met fr gudziak come back kyiv visitalso john t...
141             word back chinese availabilityblair fine
925                                       steinberg call
827                                       august hillary
655    miss maria deniseaddress ave rue abidjan cote ...
331    dear friende mre phil ipenzac provincial manag...
664    permission changing statement spokesmans name ...
249    dear friendi dr yetunde 

In [208]:
n=664
X_train[n]

'permission changing statement spokesmans name rather makes genericstatement us policy holds reservewe tweaked first paragraph make clear expectationon mexico consensus call worthwhile get updated call sheet accounts interveningeventsthe remaining honduras related call sheets also coming wayplease let know help anything else'

In [210]:
from nltk.corpus import wordnet

def to_wordnet_pos(treebank_tag: str):
    if not treebank_tag:
        return wordnet.NOUN
    t = treebank_tag[0]
    return {
        'J': wordnet.ADJ,   # JJ, JJR, JJS
        'V': wordnet.VERB,  # VB, VBD, VBG, VBN, VBP, VBZ
        'N': wordnet.NOUN,  # NN, NNS, NNP, NNPS
        'R': wordnet.ADV    # RB, RBR, RBS
    }.get(t, wordnet.NOUN)

from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

lemma = WordNetLemmatizer()

def lemmatize_with_pos(text: str) -> str:
    tokens = word_tokenize(text)
    tagged  = pos_tag(tokens)
    lemmas = [lemma.lemmatize(tok, pos=to_wordnet_pos(tag)) for tok, tag in tagged]
    return re.sub(r"\s+", " ", " ".join(lemmas)).strip()

X_train = X_train.apply(lemmatize_with_pos)
X_train[n]

'permission change statement spokesman name rather make genericstatement u policy hold reservewe tweak first paragraph make clear expectationon mexico consensus call worthwhile get update call sheet account interveningeventsthe remain honduras relate call sheet also come wayplease let know help anything else'

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
# Your code

## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code

## And the Train a Classifier?

In [ ]:
# Your code

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code